# Práctica 3: Representación del conocimiento

__Fecha de entrega: 5 de junio de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 19__

__Nombres de los estudiantes: SÁNCHEZ GÓMEZ, ALBERTO y CARPIO CUENCA, SARA__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [11]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

In [12]:
# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The wdsparql extension is already loaded. To reload it, use:
  %reload_ext wdsparql


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [13]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [14]:
%%wdsparql
SELECT ?fecha ?localidadLabel ?paisLabel
WHERE {
  wd:Q34981 wdt:P569 ?fecha; # obtenemos la fecha
            wdt:P19 ?localidad. # obtenemos localidad
  ?localidad wdt:P17 ?pais. # obtenemos el pais correspondiente a la localidad obtenida
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}

,fecha,localidadLabel,paisLabel
0,1920-01-02T00:00:00Z,Petrovichi,Russia


Hemos usado primero la propiedad "fecha de nacimiento" para obtener la fecha y la propiedad "lugar de nacimiento" para obtener la localidad. Luego hemos obtenido el país al que pertenece la localidad obtenida con la propiedad "país".

### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [15]:
%%wdsparql
SELECT ?ocupacionLabel
WHERE {
  wd:Q34981 wdt:P106 ?ocupacion. # obtenemos ocupaciones de Asimov
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
ORDER BY ASC(?ocupacionLabel)

,ocupacionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


Hemos usado la propiedad "ocupación" para obtener todas sus profesiones y luego las hemos ordenado de forma ascendente con ORDER BY, usando la ocupacionLabel.

### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [16]:
%%wdsparql
SELECT ?ocupacionLabel
WHERE {
  wd:Q34981 wdt:P106 ?ocupacion. # obtenemos las ocupaciones
  ?ocupacion wdt:P279* wd:Q36180. # nos quedamos con las que pertenecen a subclases de Escritor
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
ORDER BY ASC(?ocupacionLabel)

,ocupacionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


In [17]:
%%wdsparql
SELECT ?ocupacionLabel
WHERE {
  wd:Q34981 wdt:P106 ?ocupacion.
  ?ocupacion wdt:P279 wd:Q36180.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
ORDER BY ASC(?ocupacionLabel)

,ocupacionLabel
0,journalist
1,non-fiction writer
2,novelist
3,prosaist
4,science writer
5,screenwriter


Vemos que hijos directos de la clase `Escritor` solo son 6 (que son los de la segunda consulta). En cambio, si miramos la jerarquía completa de descendencia, vemos que todas sus profesiones son subclases de `Escritor`.

### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [18]:
%%wdsparql
SELECT ?esposaLabel ?fechaInicio ?fechaFinal
WHERE 
{
  wd:Q34981 p:P26 [ps:P26 ?esposa;
                   pq:P580 ?fechaInicio;
                   pq:P582 ?fechaFinal].
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}

,esposaLabel,fechaInicio,fechaFinal
0,Gertrude Asimov,1942-07-26T00:00:00Z,1973-01-01T00:00:00Z
1,Janet Asimov,1973-01-01T00:00:00Z,1992-01-01T00:00:00Z


Tenemos que acceder al nodo de información sobre los matrimonios de Asimov. Para ello, usamos el prefijo "p". Una vez dentro del nodo, accedemos al nombre de su esposa con el prefijo "ps".
Para saber la información sobre las fechas, accedemos a estos campos con el prefijo "pq".

### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [19]:
%%wdsparql
SELECT ?premioLabel ?fecha ?obraLabel
WHERE 
{
  wd:Q34981 p:P166 ?statement.
  ?statement ps:P166 ?premio.
  OPTIONAL {?statement pq:P585 ?fecha.}
  OPTIONAL {?statement pq:P1686 ?obra.}
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}
ORDER BY (?fecha)

,premioLabel,fecha,obraLabel
0,Fellow of the Committee for Skeptical Inquiry,None,None
1,AAAS Fellow,None,None
2,Hugo Award,1963-01-01T00:00:00Z,None
3,James T. Grady-James H. Stack Award for Interp...,1965-01-01T00:00:00Z,None
4,Hugo Award,1966-01-01T00:00:00Z,Foundation Trilogy
5,Edward E. Smith Memorial Award,1967-01-01T00:00:00Z,None
6,Nebula Award for Best Novel,1972-01-01T00:00:00Z,The Gods Themselves
7,Hugo Award for Best Novel,1973-01-01T00:00:00Z,The Gods Themselves
8,Locus Award for Best Novel,1973-01-01T00:00:00Z,The Gods Themselves
9,Klumpke-Roberts Award,1975-01-01T00:00:00Z,None


Usamos la misma idea que en la anterior consulta, solo que ahora tenemos que hacer el uso del OPTIONAL.
Tenemos que usarlo para cada una de las tripletas relacionadas con la fecha y la obra, ya que si lo ponemos todas juntas nos devuelve solamente los premios donde los dos campos están disponibles.
Para las consultas donde el campo no esta disponible no se muestra nada.

### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [20]:
%%wdsparql
SELECT ?obraLabel
WHERE 
{
  ?obra wdt:P50 wd:Q34981;
        wdt:P136 [wdt:P279* wd:Q3238422].
 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}
ORDER BY ?obraLabel

,obraLabel
0,. . . That Thou Art Mindful of Him
1,2430 A. D.
2,A Loint of Paw
3,A Statue for Father
4,Anniversary
5,Author! Author!
6,Black Friar of the Flame
7,Blank!
8,Blind Alley
9,Breeds There a Man...?


Lo primero que hacemos es obtener todas las obras escritas por Asimov. Una vez que las tenemos, nos quedamos solamente con las que su género coincide con alguno de los subgéneros de "obras literarias de ciencia ficción". Estos subgéneros los generamos como variables anónimas en la parte entre "[ ]"

### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [21]:
%%wdsparql
SELECT ?obraLabel (MIN(?fecha) AS ?primeraFecha)
WHERE 
{
  ?obra wdt:P50 wd:Q34981;
        wdt:P136 [wdt:P279* wd:Q3238422].
  OPTIONAL {?obra wdt:P577 ?fecha}.
 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}
GROUP BY ?obraLabel
ORDER BY ?primeraFecha

,obraLabel,primeraFecha
0,Lucky Starr series,None
1,Foundation series,None
2,Satisfaction Guaranteed,None
3,Foundation Trilogy,None
4,The Weapon Too Dreadful to Use,1939-01-01T00:00:00Z
5,Trends,1939-01-01T00:00:00Z
6,Marooned off Vesta,1939-03-01T00:00:00Z
7,Half-Breeds on Venus,1940-01-01T00:00:00Z
8,Ring Around the Sun,1940-01-01T00:00:00Z
9,The Callistan Menace,1940-01-01T00:00:00Z


Simplemente, usamos la consulta anterior, pero añadiendo la tripleta para calcular la fecha de publicación de cada obra (dentro de un bloque OPTIONAL por si acaso no dispone de este atributo).
Como cada obra puede estar publicada en diferentes fechas, las agrupamos por su nombre solamente, y nos quedamos con la mínima (la primera) con una función de agregación MIN.
Por último lo ordenamos por fecha para que salga con orden cronológico.

### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [24]:
%%wdsparql
SELECT (COUNT(?obra) AS ?count)
WHERE
{
  ?obra wdt:P50 wd:Q34981;
        wdt:P31 wd:Q7725634.
}

,count
0,272


Seleccionamos las obras que tengan por autor a Asimov y que sean instancias de obra literaria. Con la función de agregación COUNT, hacemos el conteo.

### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [26]:
%%wdsparql
SELECT DISTINCT ?obraLabel ?fecha
WHERE {
  ?obra wdt:P50 wd:Q34981;
        wdt:P577 ?fecha.
  FILTER(("1970-01-01"^^xsd:dateTime <= ?fecha) && (?fecha < "1980-01-01"^^xsd:dateTime))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en". }
}
ORDER BY (?fecha)

,obraLabel,fecha
0,Waterclap,1970-01-01T00:00:00Z
1,2430 A. D.,1970-01-01T00:00:00Z
2,Asimov's Guide to Shakespeare,1970-01-01T00:00:00Z
3,The Solar System and Back,1970-01-19T00:00:00Z
4,As Chemist to Chemist,1970-05-01T00:00:00Z
5,The Stars in their Courses,1971-01-01T00:00:00Z
6,The Gods Themselves,1972-01-01T00:00:00Z
7,Mirror Image,1972-01-01T00:00:00Z
8,The Early Asimov,1972-01-01T00:00:00Z
9,Kid Stuff,1972-01-01T00:00:00Z


Recuperamos las obras escritas por Asimov y sus fechas de publicación. Con la clausula filter seleccionamos aquellas obras cuyas fechas de publicación estén entre 1970 y 1980. Con el modificador de búsqueda DISTINCT nos aseguramos de que cada obra salga una sola vez. Adicionalmente, las ordenamos por fecha para una mejor visualización.

### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

In [27]:
%%wdsparql
SELECT ?centroEducativoLabel ?autorLabel ?fechaNacimiento ?fechaMuerte
WHERE {  
  wd:Q34981 wdt:P69 ?centroEducativo.
  ?autor wdt:P106 wd:Q18844224;
         wdt:P69 ?centroEducativo.
  FILTER ( ?autor not in ( wd:Q34981 ) )
  OPTIONAL {?autor wdt:P569 ?fechaNacimiento.}
  OPTIONAL {?autor wdt:P570 ?fechaMuerte.}
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE], en". }
}
ORDER BY ?centroEducativoLabel ?autorLabel

,centroEducativoLabel,autorLabel,fechaNacimiento,fechaMuerte
0,Columbia University,Alaya Dawn Johnson,1982-03-31T00:00:00Z,None
1,Columbia University,Algis Budrys,1931-01-09T00:00:00Z,2008-06-09T00:00:00Z
2,Columbia University,Arthur W. Saha,1923-10-31T00:00:00Z,1999-11-19T00:00:00Z
3,Columbia University,Ben Parris,1961-01-01T00:00:00Z,None
4,Columbia University,David G. Hartwell,1941-07-10T00:00:00Z,2016-01-20T00:00:00Z
5,Columbia University,David Kyle,1919-02-14T00:00:00Z,2016-09-18T00:00:00Z
6,Columbia University,Donald Barr,1921-08-08T00:00:00Z,2004-02-05T00:00:00Z
7,Columbia University,Doris Pitkin Buck,1898-01-03T00:00:00Z,1980-12-04T00:00:00Z
8,Columbia University,Dorothy B. Hughes,1904-08-10T00:00:00Z,1993-05-06T00:00:00Z
9,Columbia University,Eric Temple Bell,1883-02-07T00:00:00Z,1960-12-21T00:00:00Z


Primero, buscamos todos los centros educativos en los que estudió Asimov. Luego, buscamos a todos los autores de ciencia ficción que hayan estudiado en alguno de esos centros. Eliminamos a Asimov de la lista de autores con FILTER. Por último, buscamos las fechas de nacimiento y defunción de cada uno de los autores restante usando OPTIONAL, ya que es posible que falte alguna de las fechas.

### Consulta 11

¿Cuántos asertos hay sobre Asimov en Wikidata? Ten en cuenta que Asimov puede aparece tanto como sujeto como objeto de cada tripleta.

In [28]:
%%wdsparql
SELECT ?suma
WHERE
{
  {
    SELECT (COUNT(?p1) AS ?count1)
    WHERE{
      wd:Q34981 ?p1 ?objeto.
    }
  }
  {
    SELECT (COUNT(?p2) AS ?count2)
    WHERE{
      ?subejto ?p2 wd:Q34981.
    }
  }
    BIND(?count1 + ?count2 AS ?suma)
}

,suma
0,2277


Realizamos las dos consultas por separado y calculamos la suma.

__Fecha de las consultas: 21 de abril de 2022__

In [ ]:
## Desinstalar wdsparql
import sys

!{sys.executable} -m pip uninstall -y wdsparql